In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
xls = pd.ExcelFile('ProblemCData.xlsx')
energy_data = pd.read_excel(xls, 'seseds')
variables = pd.read_excel(xls, 'msncodes')

In [4]:
energy_data.head(100)

,MSN,StateCode,Year,Data
0,ABICB,AZ,1960,0.00000
1,ABICB,AZ,1961,0.00000
2,ABICB,AZ,1962,0.00000
3,ABICB,AZ,1963,0.00000
4,ABICB,AZ,1964,0.00000
5,ABICB,AZ,1965,0.00000
6,ABICB,AZ,1966,0.00000
7,ABICB,AZ,1967,0.00000
8,ABICB,AZ,1968,0.00000
9,ABICB,AZ,1969,0.00000


In [5]:
variables.head(100)

,MSN,Description,Unit,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,ABICB,Aviation gasoline blending components consumed...,Billion Btu,NaN,NaN,NaN,NaN
1,ABICP,Aviation gasoline blending components consumed...,Thousand barrels,NaN,NaN,NaN,NaN
2,ARICB,Asphalt and road oil consumed by the industria...,Billion Btu,NaN,NaN,NaN,NaN
3,ARICD,Asphalt and road oil price in the industrial s...,Dollars per million Btu,NaN,NaN,NaN,NaN
4,ARICP,Asphalt and road oil consumed by the industria...,Thousand barrels,NaN,NaN,NaN,NaN
5,ARICV,Asphalt and road oil expenditures in the indus...,Million dollars,NaN,NaN,NaN,NaN
6,ARTCB,Asphalt and road oil total consumption.,Billion Btu,NaN,NaN,NaN,NaN
7,ARTCD,"Asphalt and road oil average price, all sectors.",Dollars per million Btu,NaN,NaN,NaN,NaN
8,ARTCP,Asphalt and road oil total consumption.,Thousand barrels,NaN,NaN,NaN,
9,ARTCV,Asphalt and road oil total expenditures.,Million dollars,NaN,NaN,NaN,NaN


In [8]:
energy_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105744 entries, 0 to 105743
Data columns (total 4 columns):
MSN          105744 non-null object
StateCode    105744 non-null object
Year         105744 non-null int64
Data         105744 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.2+ MB


In [10]:
variable_columns = energy_data['MSN'].unique()

In [11]:
variable_columns

array(['ABICB', 'ABICP', 'ARICB', 'ARICD', 'ARICP', 'ARICV', 'ARTCB',
       'ARTCD', 'ARTCP', 'ARTCV', 'ARTXB', 'ARTXD', 'ARTXP', 'ARTXV',
       'AVACB', 'AVACD', 'AVACP', 'AVACV', 'AVTCB', 'AVTCD', 'AVTCP',
       'AVTCV', 'AVTXB', 'AVTXD', 'AVTXP', 'AVTXV', 'BMTCB', 'CLACB',
       'CLACD', 'CLACK', 'CLACP', 'CLACV', 'CLCCB', 'CLCCD', 'CLCCP',
       'CLCCV', 'CLEIB', 'CLEID', 'CLEIK', 'CLEIP', 'CLEIV', 'CLHCK',
       'CLICB', 'CLICD', 'CLICP', 'CLICV', 'CLKCB', 'CLKCD', 'CLKCK',
       'CLKCP', 'CLKCV', 'CLOCB', 'CLOCD', 'CLOCK', 'CLOCP', 'CLOCV',
       'CLOSB', 'CLPRB', 'CLPRK', 'CLPRP', 'CLRCB', 'CLRCD', 'CLRCP',
       'CLRCV', 'CLRFB', 'CLTCB', 'CLTCD', 'CLTCP', 'CLTCV', 'CLTXB',
       'CLTXD', 'CLTXP', 'CLTXV', 'CLXCD', 'CLXCV', 'COICB', 'COICP',
       'COPRK', 'DFACB', 'DFACD', 'DFACP', 'DFACV', 'DFCCB', 'DFCCD',
       'DFCCP', 'DFCCV', 'DFEID', 'DFEIV', 'DFICB', 'DFICD', 'DFICP',
       'DFICV', 'DFISB', 'DFRCB', 'DFRCD', 'DFRCP', 'DFRCV', 'DFRFB',
       'DFTCB', 'DFT

In [12]:
energy_data['StateCode'].unique()

array(['AZ', 'CA', 'NM', 'TX'], dtype=object)

In [13]:
AZ_data = energy_data[energy_data['StateCode'] == 'AZ']
CA_data = energy_data[energy_data['StateCode'] == 'CA']
NM_data = energy_data[energy_data['StateCode'] == 'NM']
TX_data = energy_data[energy_data['StateCode'] == 'TX']

In [14]:
AZ_data

,MSN,StateCode,Year,Data
0,ABICB,AZ,1960,0.00000
1,ABICB,AZ,1961,0.00000
2,ABICB,AZ,1962,0.00000
3,ABICB,AZ,1963,0.00000
4,ABICB,AZ,1964,0.00000
5,ABICB,AZ,1965,0.00000
6,ABICB,AZ,1966,0.00000
7,ABICB,AZ,1967,0.00000
8,ABICB,AZ,1968,0.00000
9,ABICB,AZ,1969,0.00000


In [15]:
AZ_data_reformatted = pd.DataFrame(columns = variable_columns)
CA_data_reformatted = pd.DataFrame(columns = variable_columns)
NM_data_reformatted = pd.DataFrame(columns = variable_columns)
TX_data_reformatted = pd.DataFrame(columns = variable_columns)

In [19]:
for column in variable_columns:
    AZ_data_reformatted[column] = AZ_data[AZ_data['MSN'] == column]['Data']
    CA_data_reformatted[column] = CA_data[CA_data['MSN'] == column]['Data']
    NM_data_reformatted[column] = NM_data[NM_data['MSN'] == column]['Data']
    TX_data_reformatted[column] = TX_data[TX_data['MSN'] == column]['Data']
    

In [18]:
AZ_data_reformatted.head()

,ABICB,ABICP,ARICB,ARICD,ARICP,ARICV,ARTCB,ARTCD,ARTCP,ARTCV,...,WWTXB,WWTXD,WWTXV,WXICB,WXICD,WXICP,WXICV,WYEGB,WYEGP,WYTCB
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
AZ_data_reformatted['Year'] = AZ_data['Year'].unique()

In [21]:
AZ_data_reformatted.head()

,ABICB,ABICP,ARICB,ARICD,ARICP,ARICV,ARTCB,ARTCD,ARTCP,ARTCV,...,WWTXD,WWTXV,WXICB,WXICD,WXICP,WXICV,WYEGB,WYEGP,WYTCB,Year
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1960
1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1961
2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1962
3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1963
4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1964
